# Reading XML Data

Workbook created by Martin Gossow. This code follows the other two reading notebooks.

### Driving Question

Develop a pipeline that allows us to input data from Apple Health's app, which some of the participants used. The data exported in XML format, so we need a different way to process the data. As usual though, we want to change the format into the standard form used in the other two booklets.

### Setting up Packages and Filename

In [1]:
#Import required packages
import numpy as np
import pandas as pd
from datetime import datetime

import xmltodict #This will need to be installed in anaconda.

In [2]:
#File 
FILEPATH = "../../data/Participant_ID_02/export.xml"

### Reading the Data

We use the *xmltodict* package to read in the data, which we will then convert into a csv file. The reading of the data borrows a lot of code from a blogpost by Guido Casiraghi, linked [here](https://medium.com/better-programming/analyze-your-icloud-health-data-with-pandas-dd5e963e902f).

In [3]:
with open(FILEPATH, 'r') as xml_file:
    input_data = xmltodict.parse(xml_file.read())

Now that we have the input data, we can extract the relevant information.

In [4]:
record_list = input_data['HealthData']['Record']
df = pd.DataFrame(record_list)
df.columns

Index(['@type', '@sourceName', '@sourceVersion', '@device', '@unit',
       '@creationDate', '@startDate', '@endDate', '@value'],
      dtype='object')

For the standard data form, we only need the step count (given in @value), and the start and end date, from which we can extract the necessary datetime data.

In [5]:
#Convert the dates to datetime objects
date_format = '%Y-%m-%d %H:%M:%S %z'
df['@startDate'] = pd.to_datetime(df['@startDate'],
                                    format = date_format)
df['@endDate'] = pd.to_datetime(df['@endDate'],
                                    format = date_format)
df['@value'] = pd.to_numeric(df['@value'])

We then extract the three relevant columns and rename the columns for ease of use.

In [6]:
dat = df[['@startDate','@endDate','@value']]
dat.columns = ["Start", "End", "Steps"]
dat.head(20)

,Start,End,Steps
0,2017-10-10 12:18:32+11:00,2017-10-10 12:28:03+11:00,36.0
1,2017-10-10 12:28:03+11:00,2017-10-10 12:38:03+11:00,450.0
2,2017-10-10 12:38:03+11:00,2017-10-10 12:44:16+11:00,82.0
3,2017-10-10 13:04:58+11:00,2017-10-10 13:14:58+11:00,334.0
4,2017-10-10 13:14:58+11:00,2017-10-10 13:24:50+11:00,918.0
5,2017-10-10 13:24:50+11:00,2017-10-10 13:34:50+11:00,634.0
6,2017-10-10 13:34:50+11:00,2017-10-10 13:36:42+11:00,179.0
7,2017-10-10 13:43:34+11:00,2017-10-10 13:53:34+11:00,366.0
8,2017-10-10 13:53:34+11:00,2017-10-10 13:59:26+11:00,213.0
9,2017-10-10 14:04:29+11:00,2017-10-10 14:13:50+11:00,18.0


We immediately see the main issue with this data. The data is not given in standard blocks of time, so to get the number of steps in each hour we will need to do some averaging across hour boundaries. Hence, we will end up with float values for the number of steps, but this is okay since our standard data is in this form.

In [7]:
dat1 = dat.resample("H", on="Start").sum().reset_index()

We then extract the day and hour from each of these columns to put everything into standard format.

In [8]:
dat1["Date"] = dat1["Start"].dt.date
dat1["Hour"] = dat1["Start"].dt.hour
dat1 = dat1[["Date","Hour","Steps"]]
dat1.head(10)

,Date,Hour,Steps
0,2017-10-10,12,568.323630
1,2017-10-10,13,2645.670897
2,2017-10-10,14,765.451970
3,2017-10-10,15,386.225800
4,2017-10-10,16,2861.893346
5,2017-10-10,17,371.237020
6,2017-10-10,18,0.000000
7,2017-10-10,19,0.000000
8,2017-10-10,20,0.000000
9,2017-10-10,21,110.082480


### Creating a Function

As usual, we want to make a single function that reads in the dataset and outputs a dataframe containing all of the step counts in teh standard format.

In [9]:
def read_XML_data(filename):
    #Read in XML file
    with open(FILEPATH, 'r') as xml_file:
        input_data = xmltodict.parse(xml_file.read())
    #Extract record data from XML
    record_list = input_data['HealthData']['Record']
    df = pd.DataFrame(record_list)
    #Convert dates to datetime objects and steps to numeric
    date_format = '%Y-%m-%d %H:%M:%S %z'
    df['@startDate'] = pd.to_datetime(df['@startDate'], format = date_format)
    df['@endDate'] = pd.to_datetime(df['@endDate'], format = date_format)
    df['@value'] = pd.to_numeric(df['@value'])
    #Sum up values for each hour
    dat = df.resample("H", on="@startDate").sum().reset_index()
    #Extract date and hour information, and relabel columns
    dat["Date"] = dat["@startDate"].dt.date
    dat["Hour"] = dat["@startDate"].dt.hour
    dat["Steps"] = dat["@value"]
    dat = dat[["Date","Hour","Steps"]]
    
    return dat

In [10]:
read_XML_data(FILEPATH).head(10)

,Date,Hour,Steps
0,2017-10-10,12,568.323630
1,2017-10-10,13,2645.670897
2,2017-10-10,14,765.451970
3,2017-10-10,15,386.225800
4,2017-10-10,16,2861.893346
5,2017-10-10,17,371.237020
6,2017-10-10,18,0.000000
7,2017-10-10,19,0.000000
8,2017-10-10,20,0.000000
9,2017-10-10,21,110.082480
